In [1]:
import pyomo.environ as pyo
import numpy as np

def create_model(first_stage_obj, second_stage_obj, scenario_data, scenario_probabilities):
    # Create the model
    model = pyo.ConcreteModel()

    # Parameters
    num_first_stage = len(first_stage_obj)
    num_second_stage = len(scenario_data[0])
    num_scenarios = len(scenario_data)

    # Sets
    model.I = pyo.RangeSet(1, num_first_stage)
    model.J = pyo.RangeSet(1, num_second_stage)
    model.Scenarios = pyo.RangeSet(0, num_scenarios - 1)

    # First stage variables
    model.x = pyo.Var(model.I, domain=pyo.Binary)

    # Second stage variables
    model.y = pyo.Var(model.J, model.I, model.Scenarios, domain=pyo.Binary)

    # Objective function
    def obj_rule(model):
        first_stage_cost = sum(first_stage_obj[i] * model.x[i] for i in model.I)
        second_stage_cost = sum(
            scenario_probabilities[s] * sum(
                second_stage_obj[j, i] * model.y[j, i, s]
                for j in model.J for i in model.I
            ) for s in model.Scenarios
        )
        return first_stage_cost + second_stage_cost

    model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    # First stage constraint
    def first_stage_constraint_rule(model):
        return sum(model.x[i] for i in model.I) <= 5

    model.first_stage_constraint = pyo.Constraint(rule=first_stage_constraint_rule)

    # Linking constraints
    def linking_constraint_rule(model, i, s):
        return sum(model.y[j, i, s] for j in model.J) <= 188 * model.x[i]

    model.linking_constraints = pyo.Constraint(model.I, model.Scenarios, rule=linking_constraint_rule)

    # Second stage constraints
    def second_stage_constraint_rule(model, j, s):
        return sum(model.y[j, i, s] for i in model.I) == scenario_data[s][j]

    model.second_stage_constraints = pyo.Constraint(model.J, model.Scenarios, rule=second_stage_constraint_rule)

    return model

def solve_model(model):
    solver = pyo.SolverFactory('glpk')
    results = solver.solve(model)
    return results

def main():
    # Sample data (replace with actual data from .cor and .sto files)
    first_stage_obj = {1: 40, 2: 60, 3: 47, 4: 68, 5: 60}
    global second_stage_obj ###########################################
    second_stage_obj = {
        (j, i): np.random.randint(-25, -1)
        for j in range(1, 26) for i in range(1, 6)
    }
    global scenario_data ##################################
    scenario_data = {
        s: {j: np.random.choice([0, 1]) for j in range(1, 26)}
        for s in range(50)
    }
    scenario_probabilities = [0.02] * 50

    # Create and solve the model
    model = create_model(first_stage_obj, second_stage_obj, scenario_data, scenario_probabilities)
    results = solve_model(model)

    # Print results
    print(results)

    # Print first-stage decisions
    for i in model.I:
        print(f"x[{i}] = {pyo.value(model.x[i])}")

if __name__ == "__main__":
    main()

solver 'glpk'


ApplicationError: No executable found for solver 'glpk'

In [ ]:
scenario_data

{0: {1: 0,
  2: 0,
  3: 0,
  4: 1,
  5: 1,
  6: 1,
  7: 0,
  8: 1,
  9: 0,
  10: 1,
  11: 0,
  12: 0,
  13: 1,
  14: 1,
  15: 1,
  16: 1,
  17: 0,
  18: 1,
  19: 1,
  20: 1,
  21: 0,
  22: 0,
  23: 0,
  24: 1,
  25: 1},
 1: {1: 1,
  2: 0,
  3: 1,
  4: 0,
  5: 1,
  6: 0,
  7: 0,
  8: 0,
  9: 1,
  10: 0,
  11: 0,
  12: 1,
  13: 0,
  14: 1,
  15: 1,
  16: 0,
  17: 1,
  18: 0,
  19: 1,
  20: 0,
  21: 1,
  22: 0,
  23: 0,
  24: 1,
  25: 0},
 2: {1: 0,
  2: 1,
  3: 1,
  4: 0,
  5: 0,
  6: 1,
  7: 1,
  8: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 0,
  13: 0,
  14: 1,
  15: 0,
  16: 1,
  17: 1,
  18: 0,
  19: 1,
  20: 1,
  21: 0,
  22: 1,
  23: 1,
  24: 1,
  25: 1},
 3: {1: 1,
  2: 1,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 1,
  9: 1,
  10: 0,
  11: 1,
  12: 0,
  13: 0,
  14: 0,
  15: 1,
  16: 0,
  17: 1,
  18: 0,
  19: 1,
  20: 0,
  21: 1,
  22: 0,
  23: 1,
  24: 1,
  25: 1},
 4: {1: 0,
  2: 0,
  3: 0,
  4: 1,
  5: 1,
  6: 0,
  7: 1,
  8: 1,
  9: 0,
  10: 0,
  11: 0,
  12: 0,
  13: 0,
  14: 0,
